Primer opción: el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno.

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
# Carga de archivo CSV
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_fusion3.csv"

# Carga el archivo CSV en un DataFrame
df_fusion3 = pd.read_csv(ruta_csv)

In [38]:
df_fusion3.head(2)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_hours,app_name,Action,Adventure,Racing,Simulation,Sports,Strategy
0,vincentsu88,42,76561198108222756,http://steamcommunity.com/id/vincentsu88,317360.0,Double Action: Boogaloo,127.0,0.04,Tupã,1,1,0,0,0,0
1,76561198070585472,4456,76561198070585472,http://steamcommunity.com/profiles/76561198070...,39680.0,The Guild II: Renaissance,0.0,0.00,Epic Mayhem,1,0,1,0,1,0


In [39]:

# Crear un diccionario de mapeo de usuarios e ítems a índices enteros
user_ids = df_fusion3['user_id'].unique()
item_ids = df_fusion3['item_id'].unique()
user_id_map = {user_id: i for i, user_id in enumerate(user_ids)}
item_id_map = {item_id: i for i, item_id in enumerate(item_ids)}

# Agregar columnas de índices enteros al DataFrame df_fusion3
df_fusion3['user_index'] = df_fusion3['user_id'].map(user_id_map)
df_fusion3['item_index'] = df_fusion3['item_id'].map(item_id_map)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(df_fusion3[['user_index', 'item_index', 'playtime_forever']], test_size=0.2)

In [40]:
# Selecciona las columnas relevantes para calcular la similitud
item_features = df_fusion3[['item_id', 'Action', 'Adventure', 'Racing', 'Simulation', 'Sports', 'Strategy']]

# Normaliza las características  aplica PCA (Análisis de Componentes Principales)
item_features_normalized = item_features / item_features.max(axis=0)

# Configura el número de componentes principales
n_components = 6  

# Aplica PCA
pca = PCA(n_components=n_components)
item_features_pca = pca.fit_transform(item_features_normalized)

# Se toma un subconjunto de ítems para reducir la carga de memoria
subset_size = 500  
item_features_subset = item_features.iloc[:subset_size]

# Calcular la matriz de similitud de coseno para el subconjunto de ítems
item_similarity = cosine_similarity(item_features_subset)

# Crear un DataFrame para la matriz de similitud 
item_similarity_df = pd.DataFrame(item_similarity, index=item_features_subset.index, columns=item_features_subset.index)

In [43]:
# Definir una función para obtener las recomendaciones para un ítem de referencia
def get_item_recommendations(item_id_referencia, item_similarity_df, num_recommendations=5):
    # Calcular la similitud de coseno entre el ítem de referencia y todos los demás ítems
    similarities = item_similarity_df[item_id_referencia]
    
    # Ordenar los ítems en función de su similitud de coseno (en orden descendente)
    recomendaciones = similarities.sort_values(ascending=False)
    
    # Eliminar el ítem de referencia de la lista de recomendaciones (si está presente)
    if item_id_referencia in recomendaciones:
        recomendaciones = recomendaciones.drop(item_id_referencia)
    
    # Tomar los primeros "num_recommendations" ítems como recomendaciones
    top_recommendations = recomendaciones.head(num_recommendations)
    
    return top_recommendations


In [44]:
# Uso de la función
item_id_referencia = 300  
num_recommendations = 5
recomendaciones = get_item_recommendations(item_id_referencia, item_similarity_df, num_recommendations)

print("Recomendaciones para el ítem de referencia", item_id_referencia, ":")
for item_id, similarity_score in recomendaciones.items():
    print("Ítem:", item_id, "- Similitud de Coseno:", similarity_score)

Recomendaciones para el ítem de referencia 300 :
Ítem: 71 - Similitud de Coseno: 0.9999999893131073
Ítem: 279 - Similitud de Coseno: 0.9999999866506519
Ítem: 198 - Similitud de Coseno: 0.9999990355175487
Ítem: 12 - Similitud de Coseno: 0.9999971323830115
Ítem: 295 - Similitud de Coseno: 0.9999968364545909
